In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import block
import ecc
import helper
import merkleblock
import network
import script
import tx

### Exercise 1

#### Check the cheat sheet for the network message structure.

#### 1.1. Parse this message
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 1.2. Make [these tests](/edit/session7/network.py) pass.
```
network.py:NetworkTest:test_parse
network.py:NetworkTest:test_serialize
```

In [2]:
# Exercise 1.1

msg = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')

# first 4 are network magic
magic = msg[:4]
# next 12 are command
command = msg[4:16]
# next 4 are payload length
payload_length = msg[16:20]
# next 4 are checksum
checksum = msg[20:24]
# rest is payload
payload = msg[24:]
print(command)

b'verack\x00\x00\x00\x00\x00\x00'


In [3]:
# Exercise 1.2

reload(network)
run_test(network.NetworkEnvelopeTest('test_parse'))
run_test(network.NetworkEnvelopeTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 2

#### 2.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:VersionMessageTest:test_serialize
```

In [4]:
# Exercise 2.1

reload(network)
run_test(network.VersionMessageTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 3

#### 3.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:GetHeadersMessageTest:test_serialize
```

#### 3.2. Make [this test](/edit/session7/network.py) pass.
```
network.py:HeadersMessageTest:test_parse
```

In [5]:
# Exercise 3.1

reload(network)
run_test(network.GetHeadersMessageTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [6]:
# Exercise 3.2

reload(network)
run_test(network.HeadersMessageTest('test_parse'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [7]:
# Handshake Example

from network import SimpleNode, VersionMessage

node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)

version = VersionMessage()
node.send(version.command, version.serialize())
print(node.wait_for_commands([b'verack']))

sending: version: 7f1101000000000000000000418da95b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d205be3d7d6e8d4788a1b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d04000000000000428da95b00000000000000000000000000000000000000000000ffff4830fd33c2e90d04000000000000000000000000000000000000000000000000e67c9a39ba308d23102f5361746f7368693a302e31362e332f6e94150001
sending: verack: 
receiving: verack: 
verack: 


### Exercise 4

#### 4.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:SimpleNodeTest:test_handshake
```

In [8]:
# Exercise 4.1

reload(network)
run_test(network.SimpleNodeTest('test_handshake'))

.
----------------------------------------------------------------------
Ran 1 test in 0.089s

OK


In [9]:
# Block Header Download Example

from block import GENESIS_BLOCK_HASH
from network import GetHeadersMessage, HeadersMessage, SimpleNode

node = SimpleNode('btc.programmingblockchain.com', testnet=False)
node.handshake()
last_block_hash = GENESIS_BLOCK_HASH
count = 1
for _ in range(20):
    getheaders = GetHeadersMessage(start_block=last_block_hash)
    node.send(getheaders.command, getheaders.serialize())
    headers_envelope = node.wait_for_commands([b'headers'])
    headers_message = HeadersMessage.parse(headers_envelope.stream())
    for b in headers_message.blocks:
        if not b.check_pow():
            raise RuntimeError('bad proof of work at block {}'.format(count))
        if last_block_hash != GENESIS_BLOCK_HASH and b.prev_block != last_block_hash:
            raise RuntimeError('discontinuous block at {}'.format(count))
        count += 1
        last_block_hash = b.hash()
        if count % 2016 == 0:
            print(b.hash().hex())
    if len(headers_message.blocks) < 2000:
        break

00000000693067b0e6b440bc51450b9f3850561b07f6d3c021c54fbd6abb9763
00000000f037ad09d0b05ee66b8c1da83030abaf909d2b1bf519c3c7d2cd3fdf
000000006ce8b5f16fcedde13acbc9641baa1c67734f177d770a4069c06c9de8
00000000563298de120522b5ae17da21aaae02eee2d7fcb5be65d9224dbd601c
000000009b0a4b2833b4a0aa61171ee75b8eb301ac45a18713795a72e461a946
00000000fa8a7363e8f6fdc88ec55edf264c9c7b31268c26e497a4587c750584
000000008ac55b5cd76a5c176f2457f0e9df5ff1c719d939f1022712b1ba2092
000000007f0c796631f00f542c0b402d638d3518bc208f8c9e5d29d2f169c084
00000000ffb062296c9d4eb5f87bbf905d30669d26eab6bced341bd3f1dba5fd
0000000074c108842c3ec2252bba62db4050bf0dddfee3ddaa5f847076b8822f
0000000067dc2f84a73fbf5d3c70678ce4a1496ef3a62c557bc79cbdd1d49f22
00000000dbf06f47c0624262ecb197bccf6bdaaabc2d973708ac401ac8955acc
000000009260fe30ec89ef367122f429dcc59f61735760f2b2288f2e854f04ac
00000000f9f1a700898c4e0671af6efd441eaf339ba075a5c5c7b0949473c80b
000000005107662c86452e7365f32f8ffdc70d8d87aa6f78630a79f7d77fbfe6
00000000984f962134a7291e3

In [10]:
# Merkle Parent Example

from helper import double_sha256

tx_hash0 = bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')

merkle_parent = double_sha256(tx_hash0+tx_hash1)
print(merkle_parent.hex())

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd


### Exercise 5

#### 5.1. Calculate the Merkle parent of these hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```

#### 5.2. Make [this test](/edit/session7/helper.py) pass
```
helper.py:HelperTest:test_merkle_parent
```

In [11]:
# Exercise 5.1

from helper import double_sha256

hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'

# bytes.fromhex to get the bin hashes
hash1 = bytes.fromhex(hex_hash1)
hash2 = bytes.fromhex(hex_hash2)
# double_sha256 the combination
parent = double_sha256(hash1+hash2)
# hex() to see the result
print(parent.hex())

7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800


In [12]:
# Exercise 5.2

reload(helper)
run_test(helper.HelperTest('test_merkle_parent'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [13]:
# Merkle Parent Level Example

from helper import double_sha256, merkle_parent
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
parent_level = []
for i in range(0, len(hex_hashes), 2):
    parent = merkle_parent(hashes[i], hashes[i+1])
    print(parent.hex())
    parent_level.append(parent)

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
3ecf6115380c77e8aae56660f5634982ee897351ba906a6837d15ebc3a225df0


### Exercise 6

#### 6.1. Calculate the next Merkle Parent Level given these hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```

#### 6.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_parent_level
```

In [14]:
# Exercise 6.1

from helper import merkle_parent

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]

# bytes.fromhex to get all the hashes in binary
hashes = [bytes.fromhex(h) for h in hex_hashes]
# if the number of hashes is odd, duplicate the last one
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
# initialize parent level
parent_level = []
# skip by two: use range(0, len(hashes), 2)
for i in range(0, len(hashes), 2):
    # calculate merkle_parent of i and i+1 hashes
    parent = merkle_parent(hashes[i], hashes[i+1])
    # print the hash's hex
    print(parent.hex())
    # add parent to parent level
    parent_level.append(parent)

26906cb2caeb03626102f7606ea332784281d5d20e2b4839fbb3dbb37262dbc1
717a0d17538ff5ad2c020bab38bdcde66e63f3daef88f89095f344918d5d4f96
d6c56a5281021a587f5a1e0dd4674bff012c69d960136d96e6d72261d5b696ae


In [15]:
# Exercise 6.2

reload(helper)
run_test(helper.HelperTest('test_merkle_parent_level'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [16]:
# Merkle Root Example

from helper import double_sha256, merkle_parent_level

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddecc9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]

current_level = [bytes.fromhex(x) for x in hex_hashes]
while len(current_level) > 1:
    current_level = merkle_parent_level(current_level)
print(current_level[0].hex())

acbcab8bcc1af95d8d563b77d24c3d19b18f1486383d75a5085c4e86c86beed6


### Exercise 7

#### 7.1. Calculate the Merkle Root given these hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 7.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_root
```

In [17]:
# Exercise 7.1

from helper import double_sha256, merkle_parent_level
hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex to get all the hashes in binary
hashes = [bytes.fromhex(h) for h in hex_hashes]
# initialize current level to be the hashes
current_level = hashes
# loop until current_level has only 1 element
while len(current_level) > 1:
    # make the current level the parent level
    current_level = merkle_parent_level(current_level)
# print the root's hex
print(current_level[0].hex())

a67772634e542799333c6c98bc903e36b652918a8d8a9e069391c55b4276c8a1


In [18]:
# Exercise 7.2

reload(helper)
run_test(helper.HelperTest('test_merkle_root'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [19]:
# Block Merkle Root Example

from helper import double_sha256, merkle_parent_level, merkle_root
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
current_level = [bytes.fromhex(x)[::-1] for x in tx_hex_hashes]
print(merkle_root(current_level)[::-1].hex())

654d6181e18e4ac4368383fdc5eead11bf138f9b7ac1e15334e4411b3c4797d9


### Exercise 8

#### 8.1. Validate the merkle root for this block on Testnet:
Block Hash:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

Merkle Root:
```
4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301
```
as seen here: https://api.blockcypher.com/v1/btc/test3/blocks/0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20

Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 8.2. Make [this test](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_validate_merkle_root
```

In [20]:
# Exercise 8.1

from helper import double_sha256, merkle_root

want = bytes.fromhex('4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301')

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex and reverse ([::-1]) to get all the hashes in binary
hashes = [bytes.fromhex(h)[::-1] for h in tx_hex_hashes]
# get the merkle root
root = merkle_root(hashes)
# see if the reversed root is the same as the wanted root
print(root[::-1] == want)

True


In [21]:
# Exercise 8.2

reload(block)
run_test(block.BlockTest('test_validate_merkle_root'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [22]:
# Merkle Tree Example

import math

total = 16

max_depth = math.ceil(math.log(total, 2))
merkle_tree = []
for depth in range(max_depth + 1):
    num_items = math.ceil(total / 2**(max_depth - depth))
    level_hashes = [None] * num_items
    merkle_tree.append(level_hashes)

for level in merkle_tree:
    print(level)

[None]
[None, None]
[None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


### Exercise 9

#### 9.1. Create a Merkle Tree with 27 items.

#### 9.2. Make [this test](/edit/session7/merkleblock.py) pass.
```
merkleblock.py:MerkleTreeTest:test_init
```

In [23]:
# Exercise 9.1

import math

total = 27

# get the max_depth (math.ceil(math.log(total, 2)))
max_depth = math.ceil(math.log(total, 2))
# initialize the tree
merkle_tree = []
# range over all possible depths (0 to max_depth INCLUSIVE)
for depth in range(max_depth + 1):
    # the number of items at this level is math.ceil(total / 2**(max_depth-depth))
    num_items = math.ceil(total / 2**(max_depth - depth))
    # initialize this level with every item being None
    level_hashes = [None] * num_items
    # add this level to the tree
    merkle_tree.append(level_hashes)

# loop over each level in the merkle tree
for level in merkle_tree:
    # print this level
    print(level)

[None, None]
[None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [24]:
# Exercise 9.2

reload(merkleblock)
run_test(merkleblock.MerkleTreeTest('test_init'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [25]:
# Merkle Tree Populating and Navigating Example

from merkleblock import MerkleTree
from helper import merkle_parent_level, merkle_parent

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
tree.nodes[3] = merkle_parent_level(tree.nodes[4])
tree.nodes[2] = merkle_parent_level(tree.nodes[3])
tree.nodes[1] = merkle_parent_level(tree.nodes[2])
tree.nodes[0] = merkle_parent_level(tree.nodes[1])
print(tree)
tree.right()
print(tree)
tree.left()
print(tree)
tree.right()
print(tree)
tree.up()
print(tree)

*597c4baf.*, 
6382df3f..., 87cf8fa3..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., 8636b7a3..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., f9dbfafc..., 

597c4baf..., 
6382df3f..., *87cf8fa3.*, 
3ba6c080..., 8e894862..., 7ab01bb6..., 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., 8636b7a3..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., f9dbfafc..., 

597c4baf..., 
6382df3f..., 87cf8fa3..., 
3ba6c080..., 8e894862..., *7ab01bb6.*, 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..

In [26]:
# Merkle Tree Populating Example #2

from merkleblock import MerkleTree
from helper import merkle_parent

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while tree.root() is None:
    if tree.is_leaf():
        tree.up()
    else:
        left_hash = tree.get_left_node()
        right_hash = tree.get_right_node()
        if left_hash is None:
            tree.left()
        elif right_hash is None:
            tree.right()
        else:
            tree.set_current_node(merkle_parent(left_hash, right_hash))
            tree.up()
print(tree)

597c4baf..., 
6382df3f..., 87cf8fa3..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., 8636b7a3..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., f9dbfafc..., 



### Exercise 10

#### 10.1. Populate this Merkle Tree given the 8 hashes
Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
```

#### 10.2. Make [this test](/edit/session7/merkleblock.py) pass.
```
merkleblock.py:MerkleTreeTest:test_populate_tree_1
```

In [27]:
# Exercise 10.1
from merkleblock import MerkleTree
from helper import merkle_parent

hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
]
# create an empty Merkle Tree
tree = MerkleTree(len(hex_hashes))
# set the bottom layer to be the binary version of the hex hashes
tree.nodes[3] = [bytes.fromhex(h) for h in hex_hashes]
# loop until the root is populated
while tree.root() is None:
    # if we have a leaf node
    if tree.is_leaf():
        # navigate up
        tree.up()
    else:
        # get the left and right hashes
        left_hash = tree.get_left_node()
        right_hash = tree.get_right_node()
        # we need both left and right to compute the current node
        if left_hash is None:
            # navigate to the left
            tree.left()
        elif right_hash is None:
            # navigate to the right
            tree.right()
        else:
            # combine left and right to set the current one
            tree.set_current_node(merkle_parent(left_hash, right_hash))
            # we're done with this sub-tree, navigate up
            tree.up()
print(tree)

64cbbc5d..., 
e2e5af20..., 5d8e7e77..., 
d7fb8a90..., 4e7a9ff0..., ee16eea4..., daab5fc2..., 
42f6f52f..., 94c3af34..., 959428d7..., a9f27b99..., 62af1100..., 76690059..., e8270fb4..., 921b8cfd..., 



In [28]:
# Exercise 10.2

reload(merkleblock)
run_test(merkleblock.MerkleTreeTest('test_populate_tree_1'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [29]:
# Merkle Tree Populating Example #3
reload(merkleblock)
from merkleblock import MerkleTree
from helper import merkle_parent

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while tree.root() is None:
    if tree.is_leaf():
        tree.up()
    else:
        left_hash = tree.get_left_node()
        if left_hash is None:
            tree.left()
        elif tree.right_exists():
            right_hash = tree.get_right_node()
            if right_hash is None:
                tree.right()
            else:
                tree.set_current_node(merkle_parent(left_hash, right_hash))
                tree.up()
        else:
            tree.set_current_node(merkle_parent(left_hash, left_hash))
            tree.up()
print(tree)

dc87b7e3..., 
6382df3f..., 6440941e..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 996be980..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., f542a085..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., 



### Exercise 11

#### 11.1. Populate this Merkle Tree given the 5 hashes
Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
```

#### 11.2. Make [this test](/edit/session7/merkleblock.py) pass.
```
merkleblock.py:MerkleTreeTest:test_populate_tree_2
```

In [30]:
# Exercise 11.1
from merkleblock import MerkleTree
from helper import merkle_parent

hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
]
# create an empty Merkle Tree
tree = MerkleTree(len(hex_hashes))
# set the bottom layer to be the binary version of the hex hashes
tree.nodes[3] = [bytes.fromhex(h) for h in hex_hashes]
# loop until the root is populated
while tree.root() is None:
    # if we have a leaf node
    if tree.is_leaf():
        # navigate up
        tree.up()
    else:
        # get the left hash
        left_hash = tree.get_left_node()
        # check to see if we have the left one
        if left_hash is None:
            # navigate to the left
            tree.left()
        elif tree.right_exists():
            # if the right one exists, then we need to combine left and right
            right_hash = tree.get_right_node()
            # check to see if the right one exists
            if right_hash is None:
                # navigate to the right one
                tree.right()
            else:
                # combine left and right
                tree.set_current_node(merkle_parent(left_hash, right_hash))
                # subtree is done, navigate up
                tree.up()
        else:
            # right doesn't exist, but left does. combine left twice
            tree.set_current_node(merkle_parent(left_hash, left_hash))
            # subtree is done, navigate up
            tree.up()
print(tree)

a8e8bd02..., 
e2e5af20..., 4e1f780d..., 
d7fb8a90..., 4e7a9ff0..., 610b17d5..., 
42f6f52f..., 94c3af34..., 959428d7..., a9f27b99..., 62af1100..., 



In [31]:
# Exercise 11.2

reload(merkleblock)
run_test(merkleblock.MerkleTreeTest('test_populate_tree_2'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
